In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [17]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_hvn.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.isocalendar().week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
stock = 'HVN'
df.tail(3)


,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,
HVN,20211208,22.5,23.50,22.45,22.95,2014400,2021-12-08,49,2021,2021-49
HVN,20211209,23.0,23.10,22.75,22.90,1069700,2021-12-09,49,2021,2021-49
HVN,20211210,23.0,24.45,23.00,23.60,4189400,2021-12-10,49,2021,2021-49


In [3]:
df = df.rename(columns={'<CloseFixed>':'<Close>'})
df = df[['<Close>','datetime','<Volume>','Period','Year_no']]

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name=stock))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
HVN,22.95,2021-12-08,2014400,2021-49,2021
HVN,22.90,2021-12-09,1069700,2021-49,2021
HVN,23.60,2021-12-10,4189400,2021-49,2021


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
HVN,37.4022,2017-01-04,8868,2017-1,2017,14.795820
HVN,40.9762,2017-01-05,764180,2017-1,2017,9.555588
HVN,36.5711,2017-01-06,250926,2017-1,2017,-10.750387


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    1236.000000
mean        0.010006
std         2.693778
min       -15.546218
25%        -1.164527
50%        -0.183655
75%         1.014714
max        14.795820
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    1236.000000
mean       30.688442
std         7.021324
min        17.800000
25%        25.187500
50%        29.200000
75%        35.750000
max        57.307600
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=800)
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                      mode ='lines', name='Span_7_Rolling_vol'))

fig.add_trace(go.Scatter(x=df.datetime, y=df['<Close>'], 
                        mode='lines+markers', name='Price'))

In [14]:
df['Volumn'].head(30).describe()

count    30.000000
mean      0.302873
std       0.498102
min       0.008868
25%       0.095791
50%       0.151806
75%       0.274057
max       2.727835
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean      0.196054
std       0.209094
min       0.008868
25%       0.111755
50%       0.144584
75%       0.167356
max       0.764180
Name: Volumn, dtype: float64

In [16]:
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return,Volumn
<Ticker>,,,,,,,
HVN,37.4022,2017-01-04,8868,2017-1,2017,14.795820,0.008868
HVN,40.9762,2017-01-05,764180,2017-1,2017,9.555588,0.764180
HVN,36.5711,2017-01-06,250926,2017-1,2017,-10.750387,0.250926
